In [1]:
# UPDATE LOG
# Iteration    Date          Changes
#-------------------------------------------------------------------------------------------------------------------------------
# 1            22-Feb-2017    First attempt at code.
#
# 2            24-Feb-2017    Added the objective of maximizing the minimum happiness.
#                             Added the objective that a meeting with *someone* is better than a meeting with no one.
#
# 3            26-Feb-2017    Changed the mapping between the choice rank and happiness points so that more points are given to the top choices.
#                             Addedd a faculty-focused output.
#                             Changed "Student Preferences" to "Visitor Preferences" for consistency.

# Potential Improvements
# - Add automatic detection and correction of NA rows.

# #Add packages as necessary
# Pkg.update()
# Pkg.add("JuMP")
# Pkg.add("Clp")
# Pkg.add("DataFrames")

# Define Helper Functions

In [2]:
function prefPoints(choice::Int64)
    # This function assigns a specific "happiness point" value to the meeting with a visitor's (choice)th choice faculty
    #
    # Input:
    #  choice = a positive integer specify the ranking of a faculty member in a student's mind.  For example, if Prof. Barton is visitor Johnny's first choice, then choice = 1.  Furthermore, if Prof. Braatz is Johnny's second choice, then choice = 2, and so on.
    #
    # Output:
    #  points = the happiness point value associated with giving Johnny a meeting with his (choice)th choice professor.
    
    # calculate the number of points
    points = (10 - choice + 1)^2
    
    return points
end

prefPoints (generic function with 1 method)

# Read in the Problem Data

In [3]:
# Read in the faculty availability
using DataFrames, DataArrays
raw = readtable("Faculty Availability.csv")

,Faculty,x11_12_Appts_,x1_3_Appts_
1,"Anderson, Dan",0,0
2,"Armstrong, Bob",0,0
3,"Barton, Paul",0,0
4,"Bazant, Martin",1,1
5,"Blankschtein, Daniel",1,1
6,"Braatz, Richard",1,1
7,"Brushett, Fikile",1,1
8,"Chakraborty, Arup",NA,1
9,"Chung, Kwanghun",NA,1
10,"Cohen, Bob",0,0


In [4]:
tail(raw)

,Faculty,x11_12_Appts_,x1_3_Appts_
1,"Swan, Jim",1,NA
2,"Tisdale, Will",1,1
3,"Trout, Bernhardt",0,0
4,"Virk, Preetinder",0,0
5,"Wang, Daniel",0,0
6,"Wittrup, Dane",1,1


In [5]:
# describe the input table
nameCol = 1
morningAvailabilityCol = 2
afternoonAvailabilityCol = 3

# count the number of faculty
n_F = size(raw,1)
F = 1:n_F

# extract the faculty namese
facultyNames = raw[:,nameCol]

# specify the number of time periods
n_T = 6
T = 1:n_T

# initialize the availability matrix
A = zeros(Int64,n_F,n_T)

# extract each faculty member's availability
for f in F
    # fill in the morning availability
    if typeof(raw[f,morningAvailabilityCol]) == Int64 && raw[f,morningAvailabilityCol] == 1
        A[f,[1 2]] = 1
    end
    
    #fill in the afternoon availability
    if typeof(raw[f,afternoonAvailabilityCol]) == Int64 && raw[f,afternoonAvailabilityCol] == 1
        A[f,[3 4 5 6]] = 1
    end
end

In [6]:
# Read in the student preferences
raw = readtable("Visitor Preferences.csv")

,Last_First,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
1,"Adams, Jeremy",Anderson,Chakraborty,Chung,Hammond,Langer,Prather,Sikes,Stephanopoulos,Wang,Wittrup
2,"Bhagchandani, Sachin",Hammond,Olsen,Cohen,Anderson,Chung,Sikes,Hatton,Love,Strano,Rutledge
3,"Biedermann, Andrew",Prather,Stephanopoulos,Love,Barton,Chung,Chakraborty,Brushett,Trout,Wittrup,Sikes
4,"Chen, Liang-Hsun",Smith,Gleason,Rutledge,Hammond,Jensen,Manthiram,Roman,Strano,Sikes,Brushett
5,"Juthani, Nidhi (PHDCEP)",Lauffenburger,Hammond,Langer,Strano,Anderson,Trout,Sikes,Prather,Chakraborty,Chung
6,"Kwon, Soonhyoung",Olsen,Strano,Gleason,Chung,Smith,Manthiram,Rutledge,Langer,Tisdale,Green
7,"Lee, Robert",Love,Prather,Sikes,Stephanopoulos,Wittrup,Anderson,Chakraborty,Hammond,Langer,Trout
8,"Nambiar, Anirudh Manoj Kumar",Jensen,Hatton,Manthiram,Roman,Smith,Trout,Doyle,Brushett,Green,Myerson
9,"Palmeri, Joseph",Langer,Anderson,Wittrup,Hammond,Blankschtein,Trout,Doyle,Swan,Love,Olsen
10,"Pujari, Srinivasa",Chung,Doyle,Armstrong,Love,Chakraborty,Sikes,Langer,Colton,Stephanopoulos,Jensen


In [7]:
tail(raw)

,Last_First,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
1,"Shen, Henry",Roman,Langer,Manthiram,Sikes,Hammond,Blankschtein,Jensen,Green,Smith,Chakraborty
2,"Song, Chen",Cohen,Doyle,Blankschtein,Strano,Gleason,Hatton,Rutledge,Smith,Sawin,Chung
3,"Wang, Yen-Ting",Gleason,Olsen,Kroll,Smith,Cohen,Rutledge,Hammond,Manthiram,Jensen,Sikes
4,"Yang, Jingfan",Bazant,Brushett,Langer,Roman,Strano,Doyle,Hammond,Jensen,Smith,Tisdale
5,"Ylitalo, Andrew",Olsen,Cohen,Chakraborty,Swan,Hammond,Stephanopoulos,Gleason,Bazant,Chung,Doyle
6,"Zeng, Joy",Strano,Jensen,Gleason,Doyle,Roman,Smith,Manthiram,Tisdale,Rutledge,Brushett


In [8]:
# describe the input table
nameCol = 1;
firstPrefCol = 2;

# count the number of volunteers
n_V = size(raw,1);
V = 1:n_V

# extract the visitors' names
visitorNames = Array{String,1}(n_V);
for v in V
    visitorNames[v] = raw[v,nameCol]
end

# initialize the preference matrix 
P = zeros(Int64,n_V,n_F)

# fill out the preference matrix
for v in V
    
    # loop over each visitor's list of names
    for c = 1:10
        # extract the name corresponding to the cth choice
        curName = raw[v,firstPrefCol + c - 1]
        
        # check for a string
        if typeof(curName) == String
        
            # find the index corresponding to the current faculty member
            f = 1
            nameFound = false
            while nameFound == false && f <= n_F
                # extract the faculty name corresponding to f
                facName = facultyNames[f]

                #compare the two strings
                if contains(facName,curName) == true # a match has been found

                    #raise the nameFound flag
                    nameFound = true

                else # no match has been found

                    #increment fInd and move on
                    f = f + 1         
                end

            end # ends the while loop

            #decide why we exited the while loop
            if nameFound == true # the name was found
                # update the preference matrix
                P[v,f] = prefPoints(c)
            else #the name was not found
                visName = visitorNames[v]
                println("Warning: Student '$visName' requested a meeting with '$curName', but I can't find this string in the faculty list.  I will skip over this. Please double-check the spelling.\n")
            end
        else # this is an empty cell
            #do nothing for this c
        end  #ends the split checking for a valid string
        
    end
end

# Define and Solve the Optimization Problem
## Initialize the Model and Define the Decision Variables

In [9]:
# Describe the Optimization Problem
using JuMP
using Cbc

m = Model()

## define the decision variables
@variable(m, x[1:n_V,1:n_F,1:n_T],Bin)
@variable(m, y[1:n_V,1:n_T],Bin)
@variable(m, h);

## Define the Constraints

In [10]:
# define the constraints
## constraint 1
for v in V, t in T
   @constraint(m, sum(x[v,:,t]) + y[v,t] == 1) 
end

## constraint 2
for f in F, t in T
   @constraint(m, sum(x[:,f,t]) <= 1) 
end

## constraint 3
for f in F, t in T
   @constraint(m, sum(x[:,f,t]) <= A[f,t]) 
end

## constraint 4
for v in V, f in F
   @constraint(m, sum(x[v,f,:]) <= 1) 
end

## constraint 5
for v in V
   @constraint(m, sum(y[v,:]) >= 1) 
end

## constraint 6
for v in V
   @constraint(m, sum(y[v,:]) <= 3) 
end

## constraint 7
for v in V
   @constraint(m, sum(x[v,f,t].*P[v,f] for f in F, t in T) >= h) 
end

# Define the Objective Function

In [11]:
# specify the weighting factors for the secondary objectives
w_meet = 1
w_min = 1

# construct the objective coefficient matrices
c_1 = zeros(n_V,n_F,n_T)
for v in V, f in F
    c_1[v,f,:] = P[v,f]
end

c_2 = zeros(n_V,n_F,n_T)
for v in V, f in F
    c_2[v,f,:] = w_meet
end

c_3 = w_min

c = c_1 + c_2

# define the objective function
@objective(m,Max,sum(sum(sum(c.*x))) + c_3*h);

## Solve the Optimization Problem

In [12]:
# Solve the Optimization Problem
status = solve(m)
println(status)

# extract the numerical solution
x = round(Int,getvalue(x))
y = round(Int,getvalue(y));


Optimal


# Export the Results
## Export Visitor-Focused Table

In [13]:
# initialize the schedule
schedule = Array{String,2}(n_V,n_T)
happiness = zeros(Float64,n_V)
firstChoice = Array{String,1}(n_V)
secondChoice = Array{String,1}(n_V)
thirdChoice = Array{String,1}(n_V)
fill!(firstChoice,"")
fill!(secondChoice,"")
fill!(thirdChoice,"")

#initialize the counters
firstChoiceMeetings = 0
secondChoiceMeetings = 0
thirdChoiceMeetings = 0

# loop over the visitors
for v in V
    # loop over the periods
    for t = 1:6
        # check for an appointment with Prof. Freetime
        if y[v,t] == 1 # visitor v is scheduled for free time during period t
            schedule[v,t] = "Prof. Freetime"
        else # look for a faculty meeting
            f = findfirst(x[v,:,t])
            if f == 0
                println("Warning: Student $v was not scheduled for a meeting or free time during period $t.\n")
            else 
                # put the faculty member f's name in the visitor's schedule
                schedule[v,t] = facultyNames[f]
                
                # check for first choice
                if c_1[v,f,t] == prefPoints(1)
                    firstChoice[v] = "Yes"
                    firstChoiceMeetings = firstChoiceMeetings + 1
                elseif c_1[v,f,t] == prefPoints(2)
                    secondChoice[v] = "Yes"
                    secondChoiceMeetings = secondChoiceMeetings + 1
                elseif c_1[v,f,t] == prefPoints(3)
                    thirdChoice[v] = "Yes"
                    thirdChoiceMeetings = thirdChoiceMeetings + 1
                end
            end
            
        end
    end
    
    #compute this visitor's happiness
    happiness[v] = sum(sum(c[v,:,:].*x[v,:,:]))/(sum(prefPoints(i) for i = 1:5) + w_meet*5)
end

In [14]:
# write the results to Excel
using DataArrays, DataFrames
visitorSchedule = DataFrame(Visitor_Name = visitorNames, Meeting_1 = schedule[:,1], Meeting_2 = schedule[:,2], Meeting_3 = schedule[:,3], Meeting_4 = schedule[:,4], Meeting_5 = schedule[:,5], Meeting_6 = schedule[:,6], Happiness_Fraction = happiness, First_Choice = firstChoice, Second_Choice = secondChoice, Third_Choice = thirdChoice)
writetable("Visitor-Focused Output.csv", visitorSchedule)

## Export the Faculty-Focused Table

In [15]:
# initialize the schedule
schedule = Array{String,2}(n_F,n_T)
fill!(schedule,"")

# loop over the faculty
for f in F
    
    # loop over the time periods
    for t in T
        
        # look for a meeting with a student
        v = findfirst(x[:,f,t])
        
        if v == 0 # no meeting was found
            # check the faculty's availability
            if A[f,t] == 1 # the faculty is available during this time slot
                #indicate that this slot is open in the faculty's schedule
                schedule[f,t] = "FREE"
            end
        else # a meeting was found
            # fill in the appropriate cell of the schedule
            schedule[f,t] = visitorNames[v]
        end
        
    end
end

# write the results to Excel
facultySchedule = DataFrame(Faculty = facultyNames, Meeting_1 = schedule[:,1], Meeting_2 = schedule[:,2], Meeting_3 = schedule[:,3], Meeting_4 = schedule[:,4], Meeting_5 = schedule[:,5], Meeting_6 = schedule[:,6])
writetable("Faculty-Focused Output.csv", facultySchedule)

# Calculate some Statistics

In [16]:
# calculate some fractions
firstChoiceFraction = firstChoiceMeetings/n_V
secondChoiceFraction = secondChoiceMeetings/n_V
thirdChoiceFraction = thirdChoiceMeetings/n_V

minHappinessFraction = round(minimum(happiness),2)
avgHappinessFraction = round(mean(happiness),2)

# print the results
println("The fraction of visitors who will meet with their first choice is: $firstChoiceFraction")
println("The fraction of visitors who will meet with their second choice is: $secondChoiceFraction")
println("The fraction of visitors who will meet with their third choice is: $thirdChoiceFraction")

println("The average happiness fraction is: $avgHappinessFraction")
println("The minimum happiness fraction is: $minHappinessFraction")

The fraction of visitors who will meet with their first choice is: 0.6666666666666666
The fraction of visitors who will meet with their second choice is: 0.5
The fraction of visitors who will meet with their third choice is: 0.6111111111111112
The average happiness fraction is: 0.66
The minimum happiness fraction is: 0.33
